## Next Word Prediction Model

## importing packages and dataset

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

## tokenize the text

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

## create input-output pairs by splitting the text into sequences of tokens

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

## pad the input sequences

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

## split the sequences into input and output

In [7]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

## convert the output to one-hot encode vectors

In [8]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## build a neural network architecture

In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## compile and train the model

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 37s 11ms/step - loss: 6.2526 - accuracy: 0.0754
Epoch 2/100
3010/3010 [==============================] - 22s 7ms/step - loss: 5.5413 - accuracy: 0.1214
Epoch 3/100
3010/3010 [==============================] - 20s 7ms/step - loss: 5.1460 - accuracy: 0.1455
Epoch 4/100
3010/3010 [==============================] - 21s 7ms/step - loss: 4.8104 - accuracy: 0.1638
Epoch 5/100
3010/3010 [==============================] - 21s 7ms/step - loss: 4.5052 - accuracy: 0.1836
Epoch 6/100
3010/3010 [==============================] - 20s 7ms/step - loss: 4.2143 - accuracy: 0.2020
Epoch 7/100
3010/3010 [==============================] - 21s 7ms/step - loss: 3.9382 - accuracy: 0.2273
Epoch 8/100
3010/3010 [==============================] - 21s 7ms/step - loss: 3.6795 - accuracy: 0.2593
Epoch 9/100
3010/3010 [==============================] - 20s 7ms/step - loss: 3.4360 - accuracy: 0.2910
Epoch 10/100
3010/3010 [==============================] - 23s 8

## generate the next word predictions

In [14]:
seed_text = "I love my"
next_words = 2

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 29ms/step
I love my dear fellow
